In [ ]:
%load_ext autoreload
%autoreload 2

import os

os.environ["LLMSCOPE_CACHE_DIR"] = "/vol/bitbucket/ad5518/llmscope_cache"

# Pipeline Test


## Dataset


In [ ]:
from llmscope.datasets.managers import AciBenchDatasetManager

aci_dataset_manager = AciBenchDatasetManager(splits=["train"])

## Task Preprocessor


In [ ]:
from datasets import Dataset

from llmscope.columns import ColumnConfig
from llmscope.tasks import TaskPreprocessor

column_config = ColumnConfig()


def preprocessing_function(dataset: Dataset) -> Dataset:
    dataset = dataset.rename_column("note", column_config.references.column_name)
    return dataset.select(range(10))


aci_task_preprocessor = TaskPreprocessor(
    name="Dialogue Summarization",
    preprocessing_function=preprocessing_function,
)

## Prompt Formatter


In [ ]:
from llmscope.prompts import PromptFormatter


def formatting_function(dialogue, **kwargs):
    system_prompt = "You are an expert clinical assistant specialising in the creation of medically accurate summaries from a dialogue between the doctor and patient."
    user_prompt = f"""Your task is to generate a clinical note based on a conversation between a doctor and a patient. Use the following format for the clinical note:

1. **CHIEF COMPLAINT**: [Brief description of the main reason for the visit]
2. **HISTORY OF PRESENT ILLNESS**: [Summary of the patient's current health status and any changes since the last visit]
3. **REVIEW OF SYSTEMS**: [List of symptoms reported by the patient]
4. **PHYSICAL EXAMINATION**: [Findings from the physical examination]
5. **RESULTS**: [Relevant test results]
6. **ASSESSMENT AND PLAN**: [Doctor's assessment and plan for treatment or further testing]

**Conversation:**
{dialogue}

**Note:**
"""
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]


aci_prompt_formatter = PromptFormatter(
    name="Generate Note", formatting_function=formatting_function
)

## LLM Manager


In [ ]:
from llmscope.llms.managers import VLlmManager

llama_llm_manager = VLlmManager(name="meta-llama/Llama-3.1-8B-Instruct")

## Evaluator


In [ ]:
from llmscope.evaluation.evaluators import BleuEvaluator

bleu_evaluator = BleuEvaluator()

## Workflow


In [ ]:
from llmscope.evaluation import ExperimentBatchConfig, TaskConfig

task_config = TaskConfig(
    dataset_manager=aci_dataset_manager,
    task_preprocessor=aci_task_preprocessor,
    prompt_formatter=aci_prompt_formatter,
)

experiment_config = ExperimentBatchConfig(
    tasks=[task_config],
    llm_managers=[llama_llm_manager],
    evaluators=[bleu_evaluator],
    column_config=column_config,
)

In [ ]:
from llmscope.workflow import Pipeline, Project

aci_project = Project(name="ACI-Bench Evaluation", reset_project=True)

aci_pipeline = Pipeline(
    experiments=experiment_config,
    project=aci_project,
)

In [ ]:
outputs = aci_pipeline.run()

In [ ]:
outputs